In [1]:
import os

import numpy as np
#import keras
import tensorflow
import glob
#from tensorflow.keras.utils import np_utils
from PIL import Image
from sklearn.model_selection import StratifiedKFold



#from tensorflow.keras.models import Sequential
#from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, KFold

# -------------------------------------------------------------------------------------
#                        初期設定部
# -------------------------------------------------------------------------------------

# GrayScaleのときに1、COLORのときに3にする
COLOR_CHANNEL = 3

# 入力画像サイズ(画像サイズは正方形とする)
INPUT_IMAGE_SIZE = 224

# 訓練時のバッチサイズとエポック数
BATCH_SIZE = 32
EPOCH_NUM = 2000

# 使用する訓練画像の入ったフォルダ(ルート)
TRAIN_PATH =  
# 使用する訓練画像の各クラスのフォルダ名
folder = ["100", "200", "250"]

# CLASS数を取得する
CLASS_NUM = len(folder)
print("クラス数 : " + str(CLASS_NUM))

#testを除いた中からvalidationの割合
validation_rate = 0.2

# 分割数を設定
k = 5

#分割交差検証の平均算出用
test_loss = [0]*k
test_mae = [0]*k
sum_mae = 0
sum_val_mae = 0
sum_test_mae = 0
sum_loss = 0
sum_val_loss = 0
sum_test_loss = 0


クラス数 : 3


In [3]:
from keras import backend as K
from keras.layers import Conv2D, Add, Input, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense
from keras.models import Model

def shortcut(conv, residual):
    conv_shape = K.int_shape(conv)
    residual_shape = K.int_shape(residual)
    if conv_shape != residual_shape:
        residual = Conv2D(filters=conv_shape[3], kernel_size=(1, 1), strides=(2, 2))(residual)
    return Add()([conv, residual])

def res_blocks(x,filter,stride=1):
    conv = Conv2D(filters=filter, kernel_size=(3, 3), strides=(stride, stride), padding="same", kernel_initializer='he_normal')(x)
    conv = BatchNormalization()(conv)
    conv = Activation("relu")(conv)
    conv = Conv2D(filters=filter, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_initializer='he_normal')(conv)
    conv = BatchNormalization()(conv)
    short_cut = shortcut(conv, x)
    conv = Activation("relu")(short_cut)
    return conv


class ResNet18:
    def __init__(self, input_shape, nb_classes):
        self.input_shape = input_shape
        self.nb_classes = nb_classes
        self.model = self.make_model()

    def make_model(self):
        inputs = Input(self.input_shape)
        x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding="same", kernel_initializer='he_normal')(inputs)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x)

        x = res_blocks(x, 64)
        x = res_blocks(x, 64)

        x = res_blocks(x, 128, stride=2)
        x = res_blocks(x, 128)

        x = res_blocks(x, 256, stride=2)
        x = res_blocks(x, 256)

        x = res_blocks(x, 512, stride=2)
        x = res_blocks(x, 512)
        
        x = GlobalAveragePooling2D()(x)
        outputs = Dense(units=self.nb_classes, activation='linear')(x)
        ResNetModel = Model(inputs=inputs, outputs=outputs)
        return ResNetModel

def build(input_shape, nb_classes):
    return ResNet18(input_shape, nb_classes).model

#model = ResNet18(input_shape=(224,224,3), nb_classes=1).model


In [ ]:
for fold_num, (train_index, test_index) in enumerate(skf.split(v_image, v_label)):
    print(f"Fold {fold_num+1}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    # 訓練データとラベル
    train_data, train_labels = v_image[train_index], v_label[train_index]
    
    # テストデータとラベル
    test_data, test_labels = v_image[test_index], v_label[test_index]
    
    #labelbinarizer
    #train_labels = label_binarizer.fit_transform(train_labels)
    #test_labels = label_binarizer.fit_transform(test_labels)
    
    #シャッフル
    train_data, train_labels = shuffle(train_data, train_labels, random_state=0)
    
    model = ResNet18(input_shape=(224,224,3), nb_classes=1).model

    # モデルのコンパイル
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

    # 訓練中に損失とMAEを記録するためのリスト
    history = model.fit(train_data, train_labels, validation_split = validation_rate, batch_size=BATCH_SIZE, epochs=EPOCH_NUM)
    # 訓練中の損失とMAEの変化をプロット
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    mae = history.history['mae']
    val_mae = history.history['val_mae']
    epochs = range(1, len(loss) + 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.yscale('log')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(epochs, mae, 'b', label='Training MAE')
    plt.plot(epochs, val_mae, 'r', label='Validation MAE')
    plt.title('Training and Validation MAE')
    plt.yscale('log')
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()
    
    score = model.evaluate(test_data, test_labels, verbose=0)
    print(len(test_data))
    test_loss[fold_num]=score[0]
    test_mae[fold_num]=score[1]
    print('Loss:', score[0])
    print('Mae:', score[1])
    
    sum_mae = sum_mae + mae[-1]
    sum_val_mae = sum_val_mae + val_mae[-1]
    sum_test_mae = sum_test_mae + score[1]
    sum_loss = sum_loss + loss[-1]
    sum_val_loss = sum_val_loss + val_loss[-1]
    sum_test_loss = sum_test_loss + score[0]
    
    # 予測値と正解の散布図を作成
    y_pred = model.predict(test_data)
    # 1次元に変換する
    #y_pred = y_pred.astype(np.int32)
    #y_pred = y_pred.flatten()
    #print("valid_labelsのデータ型:", type(test_labels[0]))
    #print("y_predのデータ型:", type(y_pred[0]))
    #print(test_labels)
    #print(len(test_labels))
    #print(y_pred)
    #print(len(y_pred))
    
    # 1次元に変換する
    #y_pred = y_pred.flatten()
    
    plt.scatter(test_labels, y_pred)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.title('True Values vs. Predictions')
    plt.show()
    
ave_mae = sum_mae/k
ave_val_mae = sum_val_mae/k
ave_test_mae = sum_test_mae/k
ave_loss = sum_loss/k
ave_val_loss = sum_val_loss/k
ave_test_loss = sum_test_loss/k

for j in range(k):
    print(f"hold{j+1}_test_loss:{test_loss[j]}")

    
for j in range(k):
    print(f"hold{j+1}_test_mae:{test_mae[j]}")
    

print(f"平均mae: {ave_mae}")
print(f"平均val_mae: {ave_val_mae}")
print(f"平均test_mae: {ave_test_mae}")
print(f"平均loss:{ave_loss}")
print(f"平均val_loss:{ave_val_loss}")
print(f"平均test_loss:{ave_test_loss}")